# FlavorFinder

Aplikacja pozwala przeglądać dostępne dania z darmowego api [mealAPI](https://www.themealdb.com), oraz dodać ulubione do lokalnej bazy danych.

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania ulubionych dań, oraz bibliotekę `Retrofit` do pobrania danych z zewnętrznego serwera.

<table><tr><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExZzlweXR6dzI1b3ZpNnk4MXo3cTJ6Yjd1NWx3ZG85ZGNzNGkyOHZobCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/NqBQ91k7zOVh5d39NK/giphy.gif" width="200" /></td><td><img src="https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExY29iNXd2Z2hiYnhldWtpeHVpeTR5YmYwM3ltZWh3bHlyYmkwcTNneCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/WiFtop3Zg3tdTUGFnh/giphy.gif" width="200" /></td><td><img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGs3ZTZmMnpsb21pdTAwbWphbGo1MmxoZGFtNmUwaGlzdmttcndwcCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/Van1cMPDSfS9haqhIL/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- Repozytorium zawiera metody dostępowe do lokalnej bazy oraz zewnętrznego api
- biblioteka `Retrofit` została wykorzystana do połączenia z zewnętrznym serwerem
- biblioteka `Coil` została wykorzystana do obsługi ładowania grafik
- `StateFlow` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania dań
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę dań pobranych z serwera
    - ekran *ulubione* pozwala przeglądać dania zapisane w lokalnej bazy danych
    - trzeci ekran umożliwia uzyskanie większej ilości informacji o wybranym daniu
- Aplikacja wykorzystuje `Jetpack Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w aktywności głównej, w celu uzyskania współdzielenia jego instancji przez wszystkie ekrany


Dodajmy wymagane zależności do projektu

#### **UWAGA** 
w tym projekcie zastosowano plugin `ksp` zamiast `kapt`

```kotlin
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath ("androidx.navigation:navigation-safe-args-gradle-plugin:2.5.3")
    }
}
plugins {
    id("com.android.application") version "8.1.1" apply false
    id("org.jetbrains.kotlin.android") version "1.9.0" apply false
    id("com.google.devtools.ksp") version "1.9.0-1.0.13" apply false
}

-----
    
plugins {
    id("com.android.application")
    id("org.jetbrains.kotlin.android")
    id ("androidx.navigation.safeargs.kotlin")
    id("com.google.devtools.ksp")
}

dependencies {

    // ROOM
    implementation("androidx.room:room-ktx:2.5.2")
    annotationProcessor("androidx.room:room-compiler:2.5.2")
    implementation("androidx.room:room-ktx:2.5.2")
    ksp("androidx.room:room-compiler:2.5.0")

    // ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")

    // Fragment
    implementation ("androidx.fragment:fragment-ktx:1.6.1")

    // Navigation
    implementation ("androidx.navigation:navigation-fragment:2.5.3")
    implementation ("androidx.navigation:navigation-ui:2.5.3")

    // Retrofit
    implementation ("com.squareup.retrofit2:retrofit:2.9.0")
    implementation ("com.google.code.gson:gson:2.9.1")
    implementation ("com.squareup.retrofit2:converter-gson:2.9.0")

    implementation("androidx.cardview:cardview:1.0.0")

    implementation ("androidx.recyclerview:recyclerview:1.3.1")

    implementation("io.coil-kt:coil:2.4.0")
    ...
}
```

dodajmy również odpowiednie upoważnienia do manifestu

```xml
<uses-permission android:name="android.permission.INTERNET"/>
```

Przejrzyjmy dane które wykorzystamy z api. W tej aplikacji pobierzemy tylko polskie dania, korzystając z endpointu https://www.themealdb.com/api/json/v1/1/filter.php?a=Polish. Zobaczmy odpowiedź.

In [ ]:
{
   "meals":[
      {
         "strMeal":"Bigos (Hunters Stew)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/md8w601593348504.jpg",
         "idMeal":"53018"
      },
      {
         "strMeal":"Go\u0142\u0105bki (cabbage roll)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/q8sp3j1593349686.jpg",
         "idMeal":"53021"
      },
      {
         "strMeal":"Paszteciki (Polish Pasties)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/c9a3l31593261890.jpg",
         "idMeal":"53017"
      },
      {
         "strMeal":"Pierogi (Polish Dumplings)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/45xxr21593348847.jpg",
         "idMeal":"53019"
      },
      {
         "strMeal":"Polskie Nale\u015bniki (Polish Pancakes)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/58bkyo1593350017.jpg",
         "idMeal":"53022"
      },
      {
         "strMeal":"Rogaliki (Polish Croissant Cookies)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/7mxnzz1593350801.jpg",
         "idMeal":"53024"
      },
      {
         "strMeal":"Ros\u00f3\u0142 (Polish Chicken Soup)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/lx1kkj1593349302.jpg",
         "idMeal":"53020"
      },
      {
         "strMeal":"Sledz w Oleju (Polish Herrings)",
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/7ttta31593350374.jpg",
         "idMeal":"53023"
      }
   ]
}

Dostaniemy nazwę, link do zdjęcia oraz identyfikator. Chcemy również dostać więcej informacji o wybranym przez użytkownika daniu, w tym celu wykorzystamy endpoint https://www.themealdb.com/api/json/v1/1/lookup.php?i=52772. Po podaniu identyfikatora otrzymamy następujące dane.

In [ ]:
{
   "meals":[
      {
         "idMeal":"52772",
         "strMeal":"Teriyaki Chicken Casserole",
         "strDrinkAlternate":null,
         "strCategory":"Chicken",
         "strArea":"Japanese",
         "strInstructions":"Preheat oven to 350\u00b0 F. Spray a 9x13-inch baking pan with non-stick spray.\r\nCombine soy sauce, ...   
         "strMealThumb":"https:\/\/www.themealdb.com\/images\/media\/meals\/wvpsxx1468256321.jpg",
         "strTags":"Meat,Casserole",
         "strYoutube":"https:\/\/www.youtube.com\/watch?v=4aZr5hZXP_s",
         "strIngredient1":"soy sauce",
         "strIngredient2":"water",
         "strIngredient3":"brown sugar",
         "strIngredient4":"ground ginger",
         "strIngredient5":"minced garlic",
         "strIngredient6":"cornstarch",
         "strIngredient7":"chicken breasts",
         "strIngredient8":"stir-fry vegetables",
         "strIngredient9":"brown rice",
         "strIngredient10":"",
         "strIngredient11":"",
         "strIngredient12":"",
         "strIngredient13":"",
         "strIngredient14":"",
         "strIngredient15":"",
         "strIngredient16":null,
         "strIngredient17":null,
         "strIngredient18":null,
         "strIngredient19":null,
         "strIngredient20":null,
         "strMeasure1":"3\/4 cup",
         "strMeasure2":"1\/2 cup",
         "strMeasure3":"1\/4 cup",
         "strMeasure4":"1\/2 teaspoon",
         "strMeasure5":"1\/2 teaspoon",
         "strMeasure6":"4 Tablespoons",
         "strMeasure7":"2",
         "strMeasure8":"1 (12 oz.)",
         "strMeasure9":"3 cups",
         "strMeasure10":"",
         "strMeasure11":"",
         "strMeasure12":"",
         "strMeasure13":"",
         "strMeasure14":"",
         "strMeasure15":"",
         "strMeasure16":null,
         "strMeasure17":null,
         "strMeasure18":null,
         "strMeasure19":null,
         "strMeasure20":null,
         "strSource":null,
         "strImageSource":null,
         "strCreativeCommonsConfirmed":null,
         "dateModified":null
      }
   ]
}

## Data

Wybierzmy tylko pola, które będą nas interesować i stwórzmy model. Dane dostaniemy w obu przypadkach jako listę, więc dodajmy klasę `MealRepsonse` zawierającą listę obiektów typu `Meal`. W klasie `Meal` definiujemy tylko pola, które nas interesują. Dla obu odpowiedzi możemy posłużyć się jednym modelem (nie ma konieczności tworzenia wielu modeli, jeżeli przynajmniej część danych się powtarza)

In [ ]:
data class MealResponse(
    val meals: List<Meal>
)

In [ ]:
@Entity(tableName = "meal")
data class Meal(
    @PrimaryKey
    val idMeal: String,
    val strArea: String,
    val strCategory: String,
    val strInstructions: String,
    val strMeal: String,
    val strMealThumb: String,
)

Dodajmy interfejs reprezentujący api, i zdefiniujmy w nim dwie metody, które zwracają wszystkie dania, oraz danie o zadanym id.

In [ ]:
interface MealApi {
    @GET("api/json/v1/1/filter.php?a=Polish")
    suspend fun getFood() : Response<MealResponse>

    @GET("api/json/v1/1/lookup.php?")
    suspend fun getFoodById(@Query("i") id: String) : Response<MealResponse>
}

Następnie dodajmy obiekt reprezentujący instancję `Retrofit`

In [ ]:
object RetrofitInstance {
    val api: MealApi by lazy {
        Retrofit.Builder()
            .baseUrl("https://www.themealdb.com/")
            .addConverterFactory(GsonConverterFactory.create())
            .build()
            .create(MealApi::class.java)
    }
}

Dodajmy `Dao` dla bazy `ROOM` ze zdefiniowanymi metodami dla dodawania, usuwania, oraz zwrócenia wszystkich elementów lokalnej bazy

In [ ]:
@Dao
interface MealDao {
    @Insert(onConflict = REPLACE)
    suspend fun insert(meal: Meal)

    @Delete
    suspend fun delete(meal: Meal)

    @Query("SELECT * FROM meal")
    fun getAllMeals() : Flow<List<Meal>>
}

Następnie dodajmy klasę abstrakcyjną reprezentującą naszą bazę danych.

In [ ]:
@Database(entities = [Meal::class], version = 1, exportSchema = false)
abstract class MealDatabase : RoomDatabase() {
    abstract fun mealDao(): MealDao

    companion object{
        @Volatile private var INSTANCE: MealDatabase? = null

        fun getDatabase(context: Context): MealDatabase {
            return INSTANCE ?: synchronized(this){
                val instance = Room.databaseBuilder(
                    context.applicationContext,
                    MealDatabase::class.java,
                    "meal_database_compose_v3"
                ).build().also { INSTANCE = it }
                instance
            }
        }
    }
}

Dodajmy repozytorium z metodami dostępowymi.

In [ ]:
class MealRepository(application: Application) {

    private val mealDao = MealDatabase.getDatabase(application).mealDao()

    suspend fun fetchData() = RetrofitInstance.api.getFood() // pobranie listy z serwera
    suspend fun fetchById(id: String) = RetrofitInstance.api.getFoodById(id) // pobranie danych o daniu o zadanym id

    val readData: Flow<List<Meal>> = mealDao.getAllMeals() // pobranie listy z lokalnej bazy
    suspend fun insert(meal: Meal) = mealDao.insert(meal) // dodanie elementu do lokalnej bazy
    suspend fun delete(meal: Meal) = mealDao.delete(meal) // usunięcie elementu z lokalnej bazy
}

## ViewModel

Wykorzystamy również wzorzec `ResourceBound` w celu ułatwienia reakcji na różne stany.

In [ ]:
sealed class Resource<T> (
    val data: T? = null,
    val message: String? = null
){
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)
    class Loading<T> : Resource<T>()
}

Rozpocznijmy implementację `ViewModel` od dodania pól

In [ ]:
class FoodViewModel(application: Application) : AndroidViewModel(application) {

    private val repository = MealRepository(application)

    private var _meals: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meals: StateFlow<Resource<MealResponse>> = _meals

    private var _meal: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meal: StateFlow<Resource<MealResponse>> = _meal

    val localMeals: StateFlow<List<Meal>> = repository.readData.stateIn(
        viewModelScope,
        SharingStarted.WhileSubscribed(),
        emptyList()
    )
    ...
}

- `meals` przechowuje listę wszystkich dań pobranych z serwera, inicjujemy stanem ładowania
- `meal` przechowuje informację o wybranym daniu pobraną z serwera, inicjujemy stanem ładowania
- `localMeals` przechowuje listę wszystkich dań dodanych do listy ulubionych i zapisanych w lokalnej bazie danych

Dodajmy metodę pomocniczą do obsługi odpowiedzi z serwera.

In [ ]:
    private fun handleMealResponse(response: Response<MealResponse>)
            : Resource<MealResponse> {
        if (response.isSuccessful)
            response.body()?.let { return Resource.Success(it) }
        return Resource.Error(response.message())
    }

- `if (response.isSuccessful)`: Ten fragment kodu sprawdza, czy odpowiedź HTTP jest udana. 
- `response.body()?.let { return Resource.Success(it) }`: Jeśli odpowiedź jest udana, to następnie używana jest funkcja `let`, która wywołuje blok kodu w przypadku, gdy `response.body()` nie jest `null`. To jest bezpieczny sposób dostępu do ciała odpowiedzi. Wewnątrz bloku tworzony jest obiekt `Resource.Success`, który zawiera ciało odpowiedzi. Ostatecznie ten obiekt jest zwracany z funkcji.
- `return Resource.Error(response.message())`: Jeśli odpowiedź nie jest udana, funkcja tworzy obiekt` Resource.Error`, który zawiera komunikat błędu uzyskany z `response.message()`. 

Następnie ten obiekt jest zwracany z funkcji.

W kolejnym kroku zaimplementujmy metody `fetchData` oraz `fetchDataById` pobierające dane z serwera.

In [ ]:
    private fun fetchData() = viewModelScope.launch {
        val response = repository.fetchData()
        delay(2000L) // wykorzystany dla pokazania stanu ładowania
        _meals.value = handleMealResponse(response)
    }
    
    fun fetchById(id: String) = viewModelScope.launch {
        val response = repository.fetchById(id)
        delay(2000L) // wykorzystany dla pokazania stanu ładowania
        _meal.value = handleMealResponse(response)
    }

Wywołujemy odpowiednią funkcję z repozytorium, następnie wykorzystujemy `delay` ab6y przedłużyć stan ładowania o 2 sekundy (pokażemy `CircularProgressBar` podczas ładowania). Ostanim krokiem w tej funkcji jest wywołanie `handleMealResponse`, która zwraca odpowiedni stan (sukces z danymi, lub błąd z odpowiednią wiadomością).

Dodajmy dwie metody obsługujące dodanie i usunięcie elementu z lokalnej bazy.

In [ ]:
    fun insert(meal: Meal) = viewModelScope.launch {
        repository.insert(meal)
    }

    fun delete(meal: Meal) = viewModelScope.launch {
        repository.delete(meal)
    }

Na koniec dodajmy jeszcze metodę sprawdzającą czy element znajduje się na liście przechowującej dane z lokalnej bazy (dzięki temu dodamy odpowiednią funkcjonalność do przycisku odpowiadającego za dodania/usunięcie elementu)

In [ ]:
    fun checkIfExistInLocalDb(meal: Meal): Boolean = meal in localMeals.value

Pełny kod `ViewModel`

In [ ]:
class MealViewModel(application: Application) : AndroidViewModel(application) {

    private val repository = MealRepository(application)

    private var _meals: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meals: StateFlow<Resource<MealResponse>> = _meals

    private var _meal: MutableStateFlow<Resource<MealResponse>> = MutableStateFlow(Resource.Loading())
    val meal: StateFlow<Resource<MealResponse>> = _meal

    val localMeals: StateFlow<List<Meal>> = repository.readData.stateIn(
        viewModelScope,
        SharingStarted.WhileSubscribed(),
        emptyList()
    )

    init {
        fetchData()
    }

    private fun fetchData() = viewModelScope.launch {
        val response = repository.fetchData()
        delay(2000L)
        _meals.value = handleMealResponse(response)
    }

    private fun handleMealResponse(response: Response<MealResponse>)
            : Resource<MealResponse> {
        if (response.isSuccessful)
            response.body()?.let { return Resource.Success(it) }
        return Resource.Error(response.message())
    }

    fun fetchById(id: String) = viewModelScope.launch {
        val response = repository.fetchById(id)
        delay(2000L)
        _meal.value = handleMealResponse(response)
    }

    fun insert(meal: Meal) = viewModelScope.launch {
        repository.insert(meal)
    }

    fun delete(meal: Meal) = viewModelScope.launch {
        repository.delete(meal)
    }

    fun checkIfExistInLocalDb(meal: Meal): Boolean = meal in localMeals.value
}

## Interfejs użytkownika

Wykorzystamy `BottomNavigation`, na którym umieścimy dwa ekrany - wyświetlający listę pobraną z serwera, oraz listę pobraną z lokalnej bazy. Dodamy również trzeci ekran (poza `BottomNavigation`) wyświetlający widok szczegółowy.

Dodajmy nawigację

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:id="@+id/navigation"
    app:startDestination="@id/foodListFragment">

    <fragment
        android:id="@+id/detailFragment"
        android:name="com.example.flavorfinderkotlin.ui.fragments.DetailFragment"
        android:label="DetailFragment" >
        <action
            android:id="@+id/action_detailFragment_to_foodListFragment"
            app:destination="@id/foodListFragment" />
        <action
            android:id="@+id/action_detailFragment_to_favoriteFragment"
            app:destination="@id/favoriteFragment" />
    </fragment>
    <fragment
        android:id="@+id/favoriteFragment"
        android:name="com.example.flavorfinderkotlin.ui.fragments.FavoriteFragment"
        android:label="FavoriteFragment" >
        <action
            android:id="@+id/action_favoriteFragment_to_detailFragment"
            app:destination="@id/detailFragment" />
    </fragment>
    <fragment
        android:id="@+id/foodListFragment"
        android:name="com.example.flavorfinderkotlin.ui.fragments.FoodListFragment"
        android:label="FoodListFragment" >
        <action
            android:id="@+id/action_foodListFragment_to_detailFragment"
            app:destination="@id/detailFragment" />
    </fragment>
</navigation>

Następnie dodajmy menu dla `BottomNavigation`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/foodListFragment"
        android:icon="@drawable/ic_food"
        android:title="Main" />
    <item
        android:id="@id/favoriteFragment"
        android:icon="@drawable/ic_favorite"
        android:title="Favorite" />
</menu>

Dodajmy layout dla głównej aktywności.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <androidx.fragment.app.FragmentContainerView
        android:id="@+id/nav_host_fragment"
        android:name="androidx.navigation.fragment.NavHostFragment"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        app:defaultNavHost="true"
        app:navGraph="@navigation/navigation"/>

    <com.google.android.material.bottomnavigation.BottomNavigationView
        android:id="@+id/bottom_nav_view"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:menu="@menu/bottom_navigation_menu" />

</LinearLayout>

Połączmy nawigację w aktywności głównej

In [ ]:
class MainActivity : AppCompatActivity() {

    private val binding: ActivityMainBinding by lazy {
        ActivityMainBinding.inflate(layoutInflater)
    }

    private val navController: NavController by lazy {
        val navHostFragment = supportFragmentManager.findFragmentById(R.id.nav_host_fragment)
                as NavHostFragment
        navHostFragment.findNavController()
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(binding.root)

        binding.bottomNavView.setupWithNavController(navController)
    }
}

Przejdźmy do layoutów fragmentów.

W layoucie przeznaczonym do wyświetlania listy pobranej z api dodamy `ProgressBar`, który będzie pokazywany w stanie `Resource.Loading`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/foodRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ProgressBar
        android:id="@+id/progressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="36dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

W layoucie przeznaczonym do wyświetlania listy pobranej z lokalnej bazy dodamy tylko `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/favoriteRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Layout wyświetlający widok szczegółowy jest nieco bardziej skopmplikowany i również zawiera `ProgressBar`. Ponadto dodamy `FAB` dzięki któremu użytkownik będzie miał możliwość dodania elementu do bazy lokalnej. Usunięcie zrealizujemy przez dodanie funkcjonalności `SwipeToDelete`. Opis elementu może nie zmieścić się na ekranie, więc całość zagnieżdżamy w `ScrollView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<ScrollView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <RelativeLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        tools:context=".DetailActivity">

        <ProgressBar
            android:id="@+id/progressBar"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignParentStart="true"
            android:layout_alignParentTop="true"
            android:layout_alignParentEnd="true"
            android:layout_gravity="center"
            android:layout_marginStart="181dp"
            android:layout_marginTop="35dp"
            android:layout_marginEnd="182dp"
            android:background="@android:color/transparent"
            android:visibility="invisible" />

        <ImageView
            android:id="@+id/foodImage"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:adjustViewBounds="true"
            android:contentDescription="" />

        <TextView
            android:id="@+id/title"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:padding="8dp"
            android:text=""
            android:theme="@style/ThemeOverlay.AppCompat.Dark" />

        <TextView
            android:id="@+id/category"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/foodImage"
            android:padding="8dp"
            android:text=""
            android:textColor="?android:textColorSecondary" />

        <TextView
            android:id="@+id/instructions"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/category"
            android:padding="8dp"
            android:text="" />

        <com.google.android.material.floatingactionbutton.FloatingActionButton
            android:id="@+id/favoriteButton"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:layout_alignParentEnd="true"
            android:layout_marginEnd="8dp"
            android:layout_marginTop="16dp"
            android:contentDescription=""
            android:src="@drawable/ic_favorite_border" />
    </RelativeLayout>
</ScrollView>

`ProgressBar` wymaga zdefiniowania layoutu dla samego *spinnera*

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:singleLine="true"
    android:text=""
    android:padding="10dp"
    android:textSize="24sp" />

Ostatnim layoutem jest element listy `RecyclerView`, skorzystamy z jednego pliku dla obu wykorzystanych list.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="vertical"
    android:layout_marginTop="16dp">

    <TextView
        android:id="@+id/name"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:text="nazwa"
        android:textSize="24sp"
        android:layout_margin="8dp"/>

    <ImageView
        android:id="@+id/image"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_height="200dp"
        android:scaleType="centerCrop"
        android:layout_marginStart="25dp"
        android:layout_marginEnd="25dp"
        android:contentDescription="" />

</LinearLayout>

Ponieważ korzystamy z `RecyclerView` musimy dodać klasy wspomagające. W tej aplikacji korzystamy z dwóch list, jednak zaimplementujemy tylko jeden adapter. Funkcję `OnClick` elementu listy przekażemy jako *lambdę*

In [ ]:
class MealComparator : DiffUtil.ItemCallback<Meal>() {
    override fun areItemsTheSame(oldItem: Meal, newItem: Meal): Boolean {
        return newItem.strMeal == oldItem.strMeal
    }

    override fun areContentsTheSame(oldItem: Meal, newItem: Meal): Boolean {
        return newItem.idMeal == oldItem.idMeal
    }
}

In [ ]:
class MealViewHolder(
    private val onClick: (String) -> Unit, // String jest identyfikatorem z klasy Meal
    private val binding: ListItemRvBinding
)
    : RecyclerView.ViewHolder(binding.root){
        fun bind(item: Meal){
            binding.name.text = item.strMeal
            binding.image.load(item.strMealThumb)
            binding.root.setOnClickListener { onClick(item.idMeal) }
        }
}

In [ ]:
class MealAdapter(
    private val onClick: (String) -> Unit,
    itemComparator: MealComparator
) : ListAdapter<Meal, MealViewHolder>(itemComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): MealViewHolder {
        return MealViewHolder(
            onClick,
            ListItemRvBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            )
        )
    }

    override fun onBindViewHolder(holder: MealViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }

    fun getItemAt(position: Int): Meal{
        return getItem(position)
    }
}

Dodajmy fragmenty

In [ ]:
class FoodListFragment : Fragment() { // pokazuje listę z api
    private lateinit var binding: FragmentFoodListBinding

    private val viewModel: MealViewModel by activityViewModels() // współdzielony viewmodel

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentFoodListBinding.inflate(inflater, container, false)
        return binding.root
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)

        val adapter = MealAdapter(
            onClick = {
                viewModel.fetchById(it) // wywołujemy metodę przed przejściem do fragmentu Detail
                                        // dzięki temu unikamy przekazania argumentu
                Navigation.findNavController(requireView()).navigate( // nawigacja do fragmentu DetailFragment
                    FoodListFragmentDirections.actionFoodListFragmentToDetailFragment()
                )

            },
            MealComparator()
        )

        viewLifecycleOwner.lifecycleScope.launch {
            viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED){
                viewModel.meals.collectLatest{ response ->
                    when (response) {
                        is Resource.Success -> { // jeżeli odpowiedź jest prawidłowa
                            hideProgressBar() // ukrywamy ProgressBar
                            response.data?.let { res ->
                                adapter.submitList(res.meals) // ładujemy dane do adaptera
                            }
                        }
                        is Resource.Error -> { // Jeżeli wystąpi błąd
                            hideProgressBar() // ukrywamy ProgressBar
                            response.message?.let { Log.e("FoodList", "Error occurred: $it") } // pokazujemy informację o błędzie
                        }
                        is Resource.Loading -> showProgressBar() // Jeżeli mamy stan ładowania, pokazujemy ProgressBar
                    }
                }
            }
        }

        binding.foodRV.apply {
            this.adapter = adapter
            layoutManager = LinearLayoutManager(requireContext())
        }
    }

    private fun hideProgressBar(){
        binding.progressBar.visibility = View.INVISIBLE
    }

    private fun showProgressBar(){
        binding.progressBar.visibility = View.VISIBLE
    }
}

In [ ]:
class FavoriteFragment : Fragment() { // pokazuje listę z ROOM
    private lateinit var binding: FragmentFavoriteBinding

    private val viewModel: MealViewModel by activityViewModels() // współdzielony viewmodel

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentFavoriteBinding.inflate(inflater, container, false)
        return binding.root
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)

        val adapter = MealAdapter(
            onClick = {
                viewModel.fetchById(it) // wywołujemy metodę przed przejściem do fragmentu Detail
                                        // dzięki temu unikamy przekazania argumentu
                Navigation.findNavController(requireView()).navigate( // nawigacja do fragmentu DetailFragment
                    FavoriteFragmentDirections.actionFavoriteFragmentToDetailFragment()
                )

            },
            MealComparator()
        )

        viewLifecycleOwner.lifecycleScope.launch {
            viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED){
                viewModel.localMeals.collectLatest{ meals -> adapter.submitList(meals) } // ładowanie danych do adaptera
            }
        }

        binding.favoriteRV.apply {
            this.adapter = adapter
            layoutManager = LinearLayoutManager(requireContext())
        }

        swipeToDelete(adapter)
    }

    private fun swipeToDelete(adapter: MealAdapter) { // usunięcie elementu przez przeciągnięcie do lewej lub prawej
        ItemTouchHelper(object : ItemTouchHelper.SimpleCallback(
            0,
            ItemTouchHelper.RIGHT or ItemTouchHelper.LEFT
        ) {
            override fun onMove(
                recyclerView: RecyclerView,
                viewHolder: RecyclerView.ViewHolder,
                target: RecyclerView.ViewHolder
            ): Boolean {
                return false
            }

            override fun onSwiped(viewHolder: RecyclerView.ViewHolder, direction: Int) {
                viewModel.delete(adapter.getItemAt(viewHolder.absoluteAdapterPosition))
            }
        }).attachToRecyclerView(binding.favoriteRV)
    }
}

In [ ]:
class DetailFragment : Fragment() { // pokazuje szczegółowe dane po pobraniu z api

    private lateinit var binding: FragmentDetailBinding

    private val viewModel: MealViewModel by activityViewModels()
    private val TAG = "FoodDetailFragment"

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentDetailBinding.inflate(layoutInflater, container, false)
        return binding.root
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)

        viewLifecycleOwner.lifecycleScope.launch {
            viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED){
                viewModel.meal.collectLatest{ response -> // dane znajdują się w StateFlow 'meal'
                    when (response) {
                        is Resource.Success -> { // jeżeli odpowiedź jest prawidłowa
                            hideProgressBar() // ukrywamy ProgressBar
                            response.data?.let { res ->
                                val item = res.meals.first() // dane otrzymujemy jako listę jednoelementową
                                inflate(item) // ładujemy dane na layout
                                binding.favoriteButton.setOnClickListener { // onClick FAB
                                    viewModel.insert(item) // dodanie elementu do lokalnej bazy
                                }
                                if (viewModel.checkIfExistInLocalDb(item)) // jeżeli element znajduje się w bazie
                                    binding.favoriteButton.visibility = View.INVISIBLE // ukrywam przycisk FAB
                            }
                        }
                        is Resource.Error -> { // jeżeli wystąpi błąd
                            hideProgressBar() // ukrywam ProgressBar
                            response.message?.let { Log.e(TAG, "Error occurred: $it") } // pokazujemy informację o błędzie
                        }
                        is Resource.Loading -> showProgressBar() // jeżeli mamy stan ładowania, pokazujemy ProgressBar
                    }
                }
            }
        }
    }

    private fun inflate(item: Meal) { // funkcja pomocnicza ładująca dane na layout
        binding.foodImage.load(item.strMealThumb)
        binding.category.text = item.strCategory
        binding.title.text = item.strMeal
        binding.instructions.text = item.strInstructions
    }

    private fun hideProgressBar(){
        binding.progressBar.visibility = View.INVISIBLE
    }

    private fun showProgressBar(){
        binding.progressBar.visibility = View.VISIBLE
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExZzlweXR6dzI1b3ZpNnk4MXo3cTJ6Yjd1NWx3ZG85ZGNzNGkyOHZobCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/NqBQ91k7zOVh5d39NK/giphy.gif" width="200" />